In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from knn import ParzenKNN, EuclideanMetric
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from utils import LOO_grid_search, get_distance_matrix, kernel_function

In [4]:
dataset = pd.read_csv("processed_dataset.csv", index_col=0)
dataset = dataset.drop_duplicates()  # чтобы не было двух одинаковых точек (тогда dist=0)

X = dataset.drop(columns="RainTomorrow").to_numpy()
y = dataset["RainTomorrow"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
params = {"k": [1, 2, 5, 10, 50, 75, 100], "metric_estimator": [EuclideanMetric()], "kernel_function": [kernel_function]}

best_params = LOO_grid_search(ParzenKNN, params, X_train, y_train)

100%|██████████| 1000/1000 [00:19<00:00, 51.36it/s]


In [11]:
best_params

{'best_params': {'k': 50,
  'metric_estimator': <knn.EuclideanMetric at 0x1d9d12f4cb0>,
  'kernel_function': <function utils.kernel_function(kernel_arg: 'float') -> 'float'>},
 'best_score': 0.846,
 'results': [{'params': {'k': 1,
    'metric_estimator': <knn.EuclideanMetric at 0x1d9d12f4cb0>,
    'kernel_function': <function utils.kernel_function(kernel_arg: 'float') -> 'float'>},
   'score': 0.78},
  {'params': {'k': 2,
    'metric_estimator': <knn.EuclideanMetric at 0x1d9d12f4cb0>,
    'kernel_function': <function utils.kernel_function(kernel_arg: 'float') -> 'float'>},
   'score': 0.756},
  {'params': {'k': 5,
    'metric_estimator': <knn.EuclideanMetric at 0x1d9d12f4cb0>,
    'kernel_function': <function utils.kernel_function(kernel_arg: 'float') -> 'float'>},
   'score': 0.816},
  {'params': {'k': 10,
    'metric_estimator': <knn.EuclideanMetric at 0x1d9d12f4cb0>,
    'kernel_function': <function utils.kernel_function(kernel_arg: 'float') -> 'float'>},
   'score': 0.826},
  {'par

# Инициализируем лучшую модель

In [14]:
model = ParzenKNN(k=50, metric_estimator=EuclideanMetric(), kernel_function=kernel_function)

In [18]:
from tqdm import tqdm

predictions = model.predict_bathced(X_test, X_train, y_train, n_anchor_objects=5000)

  0%|          | 0/28429 [00:00<?, ?it/s]

100%|██████████| 28429/28429 [23:44<00:00, 19.96it/s]    


In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test.astype(str), predictions)

0.83393717682648

# Сравнение с эталонным алгоритмом

In [23]:
from sklearn.neighbors import KNeighborsClassifier

sklearn_model = KNeighborsClassifier(n_neighbors=50, weights="distance")
sklearn_model.fit(X_train, y_train)

sklearn_predictions = sklearn_model.predict(X_test)

In [25]:
accuracy_score(y_test, sklearn_predictions)

0.8419923317738929

# 

# Отбор эталонных кандидатов

## Для ускорения расчетов возьмем подвыборку из обучающего датасета

In [5]:
samples_idx = np.random.choice(len(X_train), 10000, replace=False)
X_train = X_train[samples_idx]
y_train = y_train[samples_idx]

In [20]:
from utils import get_compact_profile

In [ ]:
get_compact_profile(100, X_train, y_train, None, samples_for_estimation=1000)

np.float64(0.249)